<a href="https://colab.research.google.com/github/rixprakash/DS2002F24/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/rixprakash/DS2002F24

Cloning into 'DS2002F24'...
remote: Enumerating objects: 366, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 366 (delta 126), reused 101 (delta 64), pack-reused 167 (from 1)
Receiving objects: 100% (366/366), 17.60 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [20]:
import pandas as pd
import json
import sqlite3
import os

# Load the CSV file
file_path = '/content/DS2002F24/dataproject1/Crime_Data.csv'
df = pd.read_csv(file_path)
dfbefore = df.copy() # Copies the original file

# Function to add the new file for Exact Date Report
def add_exact_date_report_column(df):
    try:
        # Extract the date part (first 10 characters) from 'DateReported'
        df['ExactDateReport'] = df['DateReported'].str[:10]
    except Exception as e:
        print(f"Error adding 'ExactDateReport' column: {e}")
    return df

# Function to split Officer Name
def split_reporting_officer(df):
    try:
        # Split the 'ReportingOfficer' column by the comma
        df[['OfficerLastName', 'OfficerFirstName']] = df['ReportingOfficer'].str.split(', ', expand=True)
    except Exception as e:
        print(f"Error splitting 'ReportingOfficer' into 'OfficerFirstName' and 'OfficerLastName': {e}") # If an error occurs
    return df

# Function to convert CSV file to JSON
def convert_csv_to_json(df, file_path):
    try:
        json_data = df.to_json(orient='records')
        json_file_path = file_path.replace('.csv', '.json') # Changing to json
        with open(json_file_path, 'w') as json_file:
            json_file.write(json_data)
        print(f"CSV file converted to JSON and saved as {json_file_path}")
    except Exception as e:
        print(f"Error converting CSV to JSON: {e}")

# Function to convert CSV file to SQL
def convert_csv_to_sql(df, file_path, db_name='crime_data.db', table_name='crime_records'):
    try:
        # Connect to SQLite database (or create it if it doesn't exist)
        conn = sqlite3.connect(db_name)

        # Write the dataframe to the SQL database
        df.to_sql(table_name, conn, if_exists='replace', index=False) # Stores as a SQL database
        conn.close()
        print(f"CSV file converted to SQL and saved in {db_name} database, table: {table_name}")
    except Exception as e:
        print(f"Error converting CSV to SQL: {e}")

# Function to summarize data
def summarize_data(df):
    print(f"Number of records: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Columns: {list(df.columns)}\n")

# Main function to let the user choose
def main():
    df_modified = add_exact_date_report_column(df)
    df_modified = split_reporting_officer(df_modified)
    choice = input("Do you want to convert the CSV file to JSON or SQL Database? Enter 'JSON' or 'SQL': ")

    if choice == 'JSON':
        convert_csv_to_json(df, file_path)
    elif choice == 'SQL':
        convert_csv_to_sql(df, file_path)
    else:
        print("Invalid choice! Please enter 'JSON' or 'SQL'.") # Prompting user to put only these two options

    print("Pre-modified Data Summary:")
    summarize_data(dfbefore)
    print("Post-modified Summary:")
    summarize_data(df_modified)

if __name__ == "__main__":
    main()


Do you want to convert the CSV file to JSON or SQL Database? Enter 'JSON' or 'SQL': JSON
CSV file converted to JSON and saved as /content/DS2002F24/dataproject1/Crime_Data.json
Pre-modified Data Summary:
Number of records: 25111
Number of columns: 9
Columns: ['RecordID', 'Offense', 'IncidentID', 'BlockNumber', 'StreetName', 'Agency', 'DateReported', 'HourReported', 'ReportingOfficer']

Post-modified Summary:
Number of records: 25111
Number of columns: 12
Columns: ['RecordID', 'Offense', 'IncidentID', 'BlockNumber', 'StreetName', 'Agency', 'DateReported', 'HourReported', 'ReportingOfficer', 'ExactDateReport', 'OfficerLastName', 'OfficerFirstName']



In [18]:
import pandas as pd
import json
import sqlite3
import os

# Set file path (hardcoded)
file_path = '/content/DS2002F24/dataproject1/sportscarprice.json'

# Function to delete 'Engine Size (L)' column from a JSON file
def delete_engine_size_column_json(file_path):
    try:
        # Load the JSON file into a DataFrame
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)
        df = pd.DataFrame(json_data)

        # Summary before processing (Ingestion Summary)
        print("Ingestion Summary:")
        print(f"Number of records: {len(df)}")
        print(f"Number of columns: {len(df.columns)}")

        # Check if the column 'Engine Size (L)' exists, and drop it
        if 'Engine Size (L)' in df.columns:
            df = df.drop(columns=['Engine Size (L)'])

        # Save the updated DataFrame back to JSON
        new_file_path = file_path.replace('.json', '_updated.json')
        df.to_json(new_file_path, orient='records', indent=4)

        # Summary after processing (Post-Processing Summary)
        print("\nPost-Processing Summary:")
        print(f"Number of records: {len(df)}")
        print(f"Number of columns: {len(df.columns)}")

        return df  # Return the modified DataFrame (optional)

    except Exception as e:
        print(f"Error processing JSON file: {e}")
        return None

# Function to convert DataFrame to CSV
def convert_json_to_csv(df, file_path):
    try:
        csv_file_path = file_path.replace('.json', '.csv')
        df.to_csv(csv_file_path, index=False)
        print(f"JSON file converted to CSV and saved as {csv_file_path}")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

# Function to convert DataFrame to SQL
def convert_json_to_sql(df, file_path, db_name='json_data.db', table_name='json_records'):
    try:
        # Connecting to the SQLite database
        conn = sqlite3.connect(db_name)

        # Write the dataframe to the SQL database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.close()
        print(f"JSON file converted to SQL and saved in {db_name} database, table: {table_name}")
    except Exception as e:
        print(f"Error converting JSON to SQL: {e}")

# Main function to allow user to choose conversion type
def main():
    # Delete Engine Size (L) column
    df_modified = delete_engine_size_column_json(file_path)

    if df_modified is not None:
        choice = input("Do you want to convert the modified JSON file to CSV or SQL Database? Enter 'CSV' or 'SQL': ") # Prompting user choice

        if choice.upper() == 'CSV':
            convert_json_to_csv(df_modified, file_path)
        elif choice.upper() == 'SQL':
            convert_json_to_sql(df_modified, file_path)
        else:
            print("Invalid choice! Please enter 'CSV' or 'SQL'.")
    else:
        print("Unable to load the JSON file. Please check the file format.") # If it can not open

if __name__ == "__main__":
    main()


Ingestion Summary:
Number of records: 1007
Number of columns: 8

Post-Processing Summary:
Number of records: 1007
Number of columns: 7
Do you want to convert the modified JSON file to CSV or SQL Database? Enter 'CSV' or 'SQL': SQL
JSON file converted to SQL and saved in json_data.db database, table: json_records


# **Reflection**
During this project, one of the main challenges I faced was managing the transformation between different file formats. Since the process of switching the input file into a JSON and SQL are different, I had to create functions that would execute based on conditions. I also had to create handle the editing/modification of the files with the data manipualtion differently. Adding/removing files from JSON and SQL are different, but fortunately CSV was straight forward to do and was similar to SQL. Specifically with these files, the transformation complex JSON structures into the relational format proved by the user seemed to be more difficult than I thought. Additionally, managing potential errors that could come into effect like missing columns or formatting issues required me to put except conditions to handle these errors allow for a smooth execution of the code. Also, I had to make sure the user put in only 2 options that were provided and if they didn't I had to not let the code execute and prompt them to put in a correct choice.

Despite these challenges, some aspects of the project were easier than I expected. Using **pandas** to handle data manipulation turned out to be pretty straightforward for csv files, and similarly with editing SQL files. Also, setting up the user input and flow control in the script was manageable to get the function to continue to use this. With this process, I allowed the user to choose between CSV or SQL output which was easy to ask, however like previously stated, to actually convert it was a little tricky. Finally, the easiest part was saving the file back to the system or saving it as a sql database. Both of these proved to be simple.

CONTINUE HERE


What proved to be harder than expected was ensuring smooth error handling across various file formats. Anticipating cases where the `"Engine Size (L)"` column might not exist, or when the JSON structure was inconsistent, required thorough validation and error messaging to ensure the pipeline handled these edge cases gracefully. Another unexpectedly complex aspect was the data conversion into SQL. SQLite databases, while convenient, have certain limitations with respect to data types, which made ensuring data integrity during conversion a bit more complicated than anticipated.

Overall, the utility developed for this project offers substantial value for future data tasks. Many data projects involve working with diverse data sources—whether they come from APIs, CSV files, or relational databases. Having a flexible ETL processor like this one can significantly streamline workflows by automating common tasks like data ingestion, format conversion, and preprocessing. This utility could easily be extended to handle more complex workflows, such as integrating data from multiple sources or adding features for more advanced data cleaning. Furthermore, in future team-based projects, this processor’s modularity, combined with version control through platforms like GitHub, would enhance collaboration and allow for incremental improvements. Overall, this ETL processor provides a solid foundation for transforming, cleaning, and converting data, making it highly useful for a wide range of future data science projects.
